First add the data functions to the path

In [ ]:
import sys
import numpy as np
import pandas as pd
import copy
from netCDF4 import Dataset
from pathlib import Path
notebook_dir = !pwd # Replace with !echo %cd% if using windows
library_dir = Path(notebook_dir[0]).parent.absolute()
sys.path.append(str(library_dir))
data_raw_path = library_dir / 'data' / 'raw'

from src.data.adcp_mission_analysis import adcp_import_data, add_dive_averages
from src.data.beam_mapping import beam2enu, beam_from_center

Use better plotting defaults

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
style_path = library_dir / 'src' / 'visualization' / 'presentation.mplstyle'
style.use(str(style_path))

Get an overview of the dataset

In [ ]:
overview, mission_dict = adcp_import_data(data_raw_path)
overview

Add beam attributes. These are taken from the fifth cell. 11.3 m from the glider on default settings.

In [ ]:
beam_attrs = add_dive_averages(overview, mission_dict)
beam_attrs
# overview.join(beam_attrs)

In [ ]:
beam_attrs.plot(subplots=True,figsize=(20,20),marker='o');

After 20 dives, the glider should be trimmed and on deep dives.

In [ ]:
foo = mission_dict['0021a']
bar = mission_dict['0021b']

In [ ]:
fig, ax = plt.subplots()
ax.scatter(-foo.glider_w_from_p,foo.vel_enu[:,0,2][:-1])
ax.set(xlim=[0,0.2], ylim=[0,0.2])

In [ ]:
mens = []
mene = []
menn = []
menu = []
for ident in list(mission_dict):
    mens.append(np.nanmean(np.abs(mission_dict[ident].vel_enu)))
    mene.append(np.nanmean(mission_dict[ident].vel_enu[:,:,0]))
    menn.append(np.nanmean(mission_dict[ident].vel_enu[:,:,1]))
    menu.append(np.nanmean(mission_dict[ident].vel_enu[:,:,2]))

In [ ]:
print(np.nanmedian(np.abs(mene)))
print(np.nanmedian(np.abs(menn)))
print(np.nanmedian(np.abs(menu)))

In [ ]:
mens2 = []
for ident in list(mission_dict):
    mens2.append(np.nanmean(mission_dict[ident].vel_beam))

In [ ]:
plt.scatter(beam_attrs.good_angle, beam_attrs.roll)

In [ ]:
plt.scatter(np.arange(len(mens)), mens2)

In [ ]:
plt.scatter(beam_attrs['heading'], mens)

In [ ]:
fig, ax = plt.subplots(4,1,figsize=(20,20))
ax = ax.ravel()
ax[0].scatter(mene,menn,label='all', c=beam_attrs['pitch'])
ax[1].scatter(beam_attrs['heading'],mene,label='east', c=beam_attrs['pitch'])
ax[2].scatter(beam_attrs['heading'],menn,label='north', c=beam_attrs['pitch'])
ax[3].scatter(beam_attrs['heading'],menu,label='up')
#ax[3].set(ylim=[-0.02,0.02])